In [3]:
!pip install python-telegram-bot==20.3 requests beautifulsoup4 transformers torch nest_asyncio

In [5]:
!pip install python-telegram-bot==20.3

In [4]:
!pip install telegram

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for telegram: filename=telegram-0.0.1-py3-none-any.whl size=1307 sha256=ce8a0aa62088a9d48db985f2d78f082f09effc53b216c89fc69d3790bba0ab93
  Stored in directory: c:\users\иван абоба\appdata\local\pip\cache\wheels\11\7a\5d\62391dcb6b9a45247192c3a711bf03ed513c14218a8b275a63
Successfully built telegram


  DEPRECATION: Building 'telegram' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'telegram'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [7]:
!pip install bs4

In [ ]:
import os
import logging
import nest_asyncio
import requests
from bs4 import BeautifulSoup
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, AutoModel
from telegram import Update
from telegram.ext import (
    ApplicationBuilder,
    CommandHandler,
    MessageHandler,
    ContextTypes,
    filters,
)

nest_asyncio.apply()

# Логирование
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)

# 1. Токенизатор и модель
TOKENIZER_NAME = "DeepPavlov/rubert-base-cased"
MODEL_NAME = "sberbank-ai/ruBert-base"
MODEL_PATH = "best_model_new.pth"

tokenizer = BertTokenizer.from_pretrained(TOKENIZER_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)

class SentimentClassifier(torch.nn.Module):
    def __init__(self, bert_model, n_classes=2, dropout=0.3):
        super().__init__()
        self.bert = bert_model
        self.dropout = torch.nn.Dropout(dropout)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        return self.classifier(self.dropout(pooled))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentimentClassifier(base_model).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# 2. Парсер новости
def extract_text(url: str) -> str:
    resp = requests.get(url, timeout=10)
    soup = BeautifulSoup(resp.content, 'html.parser')
    return "\n".join(p.get_text() for p in soup.find_all('p') if p.get_text())

# 3. Предикт
def predict_sentiment(text: str) -> str:
    enc = tokenizer(
        text, truncation=True, padding='max_length', max_length=256, return_tensors='pt'
    )
    input_ids = enc['input_ids'].to(device)
    attn_mask = enc['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        probs = F.softmax(logits, dim=1).cpu().numpy()[0]
    labels = ['Негативная', 'Позитивная']
    idx = probs.argmax()
    return f"{labels[idx]} (уверенность {probs[idx]:.2f})"

# 4. Хэндлеры Telegram
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Пришли ссылку на новость — я скажу, положительная она или отрицательная."
    )

async def handle_url(update: Update, context: ContextTypes.DEFAULT_TYPE):
    url = update.message.text.strip()
    await update.message.reply_text("Скачиваю и анализирую статью…")
    try:
        text = extract_text(url)
        if not text:
            return await update.message.reply_text("Не смог найти текст на странице.")
        result = predict_sentiment(text)
        await update.message.reply_text(f"Результат: {result}")
    except Exception as e:
        logger.error("Ошибка при обработке: %s", e)
        await update.message.reply_text("Не удалось обработать ссылку.")

def main():
    token = os.getenv("TELEGRAM_TOKEN", "токен")
    app = ApplicationBuilder().token(token).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_url))

    # Запуск polling внутри Colab
    app.run_polling()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'bs4'